In [1]:
# Importando as bibliotecas
from CoolProp.CoolProp import PropsSI, PhaseSI
import pprint
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Calcula qualquer ponto informando dicionario com duas variaveis
def calculate_point(point):
    variables = ['T', 'H', 'S', 'Q', 'P']
    input_variables = list(point.keys())
    output_variables = [variable for variable in variables if variable not in input_variables]
    output_values = PropsSI(output_variables, 
                            input_variables[0], 
                            point[input_variables[0]], 
                            input_variables[1], 
                            point[input_variables[1]], 
                            point['refrigerant']
    )

    for index, variable in enumerate(output_variables):
        point[variable] = output_values[index]

    return point

In [26]:
# Calcula ciclo baseado nas variaveis de entrada
def calculate_cycle(cycle_inputs):
    t_0 = cycle_inputs['t_external_env']

    # Point 3 (after condenser)
    point_3_saturado = {'Q': 0, 
                        'T': cycle_inputs['t_external_env'] + cycle_inputs['approach_condenser'],
                       'refrigerant': cycle_inputs['refrigerant']
                       }
    calculate_point(point_3_saturado)

    if cycle_inputs['subcooling'] == 0:
        point_3 = point_3_saturado
    else:
        point_3 = {'P': point_3_saturado['P'], 
                   'T': point_3_saturado['T'] - cycle_inputs['subcooling'],
                   'refrigerant': cycle_inputs['refrigerant']
                  }
        calculate_point(point_3)
        
    
    # Point 5 (after high temperature evaporator)    
    point_5_saturado = {'Q': 1, 
                        'T': cycle_inputs['t_internal_env_ht'] - cycle_inputs['approach_evaporator_ht'],
                        'refrigerant': cycle_inputs['refrigerant']
                       }
    calculate_point(point_5_saturado)
    
    if cycle_inputs['superheating_ht'] == 0:
        point_5 = point_5_saturado
    else:
        point_5 = {'P': point_5_saturado['P'], 
                   'T': point_5_saturado['T'] + cycle_inputs['superheating_ht'],
                   'refrigerant': cycle_inputs['refrigerant']
                  }
        calculate_point(point_5)

    # Point 8 (after low temperature evaporator)  
    point_8_saturado = {'Q': 1, 
                        'T': cycle_inputs['t_internal_env_lt'] - cycle_inputs['approach_evaporator_lt'],
                        'refrigerant': cycle_inputs['refrigerant']
                       }
    calculate_point(point_8_saturado)

    if cycle_inputs['superheating_lt'] == 0:
        point_8 = point_8_saturado
    else:
        point_8 = {'P': point_8_saturado['P'], 
                   'T': point_8_saturado['T'] + cycle_inputs['superheating_lt'],
                   'refrigerant': cycle_inputs['refrigerant']
                  }
        calculate_point(point_8)        
    
    # Point 4 (after expansion valve of HTE)  
    point_4 = {'P': point_5['P'], 
               'H': point_3['H'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_4)

    # Point 7 (after expansion valve of LTE)  
    point_7 = {'P': point_8['P'], 
               'H': point_3['H'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_7)
    
    # Point 6 (after regulator expansion valve)  
    point_6 = {'P': point_8['P'], 
               'H': point_5['H'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_6)
    
    # Point 1 (before compressor)
    point_1 = {'P': point_8['P'],
               'H': (1 - cycle_inputs['f']) * point_6['H'] + cycle_inputs['f'] * point_8['H'],
               'refrigerant': cycle_inputs['refrigerant']
    }
    calculate_point(point_1)
    
    # Point 2 (after compressor)
    point_2_isen = {'S': point_1['S'], 
                    'P': point_3['P'],
                    'refrigerant': cycle_inputs['refrigerant']
                   }
    calculate_point(point_2_isen)

    point_2 = {'P': point_3['P'], 
               'H': point_1['H'] + (point_2_isen['H'] - point_1['H']) / cycle_inputs['isentropic_efficiency'],
               'refrigerant': cycle_inputs['refrigerant']
              }
    calculate_point(point_2)
    
    # Masses
    m = cycle_inputs['work'] / (point_2['H'] - point_1['H'])
    m_lt = m * cycle_inputs['f']
    m_ht = m * (1 - cycle_inputs['f'])
    
    # COP
    q_evaporator_ht = m_ht * (point_5['H'] - point_4['H'])
    q_evaporator_lt = m_lt * (point_8['H'] - point_7['H'])
    cop = (q_evaporator_ht + q_evaporator_lt) / cycle_inputs['work'] 
    
    # Exergy Destruction
    t_0 = cycle_inputs['t_external_env']
    
    compressor_exergy_destruction = m * t_0 * (point_2['S'] - point_1['S'])
    condenser_exergy_destruction = m * t_0 * (point_3['S'] - point_2['S'] + (point_2['H'] - point_3['H']) / cycle_inputs['t_external_env'])
    expansion_valve_ht_exergy_destruction = m_ht * t_0 * (point_4['S'] - point_3['S'])
    evaporator_ht_exergy_destruction = m_ht * t_0 * (point_5['S'] - point_4['S'] - (point_5['H'] - point_4['H']) / cycle_inputs['t_internal_env_ht'])
    regulator_expansion_valve_exergy_destruction = m_ht * t_0 * (point_6['S'] - point_5['S'])
    expansion_valve_lt_exergy_destruction = m_lt * t_0 * (point_7['S'] - point_3['S'])
    evaporator_lt_exergy_destruction = m_lt * t_0 * (point_8['S'] - point_7['S'] - (point_8['H'] - point_7['H']) / cycle_inputs['t_internal_env_lt'])
    
    total_exergy_destruction = compressor_exergy_destruction + condenser_exergy_destruction \
        + expansion_valve_ht_exergy_destruction + evaporator_ht_exergy_destruction + regulator_expansion_valve_exergy_destruction \
        + expansion_valve_lt_exergy_destruction + evaporator_lt_exergy_destruction
    
    return {
        'cycle_inputs': cycle_inputs,
        'point_1': point_1,
        'point_2': point_2,
        'point_3': point_3,
        'point_4': point_4,
        'point_5': point_5,
        'point_6': point_6,
        'point_7': point_7,
        'point_8': point_8,
        'm': m,
        'm_ht': m_ht,
        'm_lt': m_lt,
        'q_evaporator_ht': q_evaporator_ht,
        'q_evaporator_lt': q_evaporator_lt,
        'cop': cop,
        'exergy_efficiency': 1 - total_exergy_destruction / cycle_inputs['work']
    }

In [10]:
# Variaveis de entrada
input_values = {
    't_external_env': 32.22 + 273.15,
    't_internal_env_ht': 4.44 + 273.15,
    't_internal_env_lt': -17.78 + 273.15,
    'approach_condenser': 0,
    'approach_evaporator_ht': 0,
    'approach_evaporator_lt': 0,
    'work': 18299.697604861303,
    'f': 0.3387218287321152,
    'isentropic_efficiency': 0.7,
    'subcooling': 5,
    'superheating_ht': 5,
    'superheating_lt': 5,
    'refrigerant': 'NH3'
}

cycle = calculate_cycle(input_values)
pprint.pprint(cycle)

{'cop': 2.8852935791667793,
 'cycle_inputs': {'approach_condenser': 0,
                  'approach_evaporator_ht': 0,
                  'approach_evaporator_lt': 0,
                  'f': 0.3387218287321152,
                  'isentropic_efficiency': 0.7,
                  'refrigerant': 'NH3',
                  'subcooling': 5,
                  'superheating_ht': 5,
                  'superheating_lt': 5,
                  't_external_env': 305.37,
                  't_internal_env_ht': 277.59,
                  't_internal_env_lt': 255.36999999999998,
                  'work': 18299.697604861303},
 'exergy_efficiency': 0.3811938000719234,
 'm': 0.04621471474880705,
 'm_ht': 0.030560782054758068,
 'm_lt': 0.01565393269404898,
 'point_1': {'H': 1616393.3717983814,
             'P': 209507.0301798974,
             'Q': -1.0,
             'S': 6468.565716701363,
             'T': 267.99108690371645,
             'refrigerant': 'NH3'},
 'point_2': {'H': 2012364.60529677,
             'P'

In [24]:
# Variaveis de entrada
input_values = {
    't_external_env': 35 + 273.15,
    't_internal_env_ht': 5 + 273.15,
    't_internal_env_lt': -15 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'work': 900,
    'f': 0.9,
    'isentropic_efficiency': 0.9,
    'subcooling': 9.439496104069462,
    'superheating_ht': 4.442792830706139,
    'superheating_lt': 0,
    'refrigerant': 'NH3'
}

cycle = calculate_cycle(input_values)
pprint.pprint(cycle)

here 0
{'cop': 3.0722141251320743,
 'cycle_inputs': {'approach_condenser': 5,
                  'approach_evaporator_ht': 5,
                  'approach_evaporator_lt': 5,
                  'f': 0.9,
                  'isentropic_efficiency': 0.9,
                  'refrigerant': 'NH3',
                  'subcooling': 9.439496104069462,
                  'superheating_ht': 4.442792830706139,
                  'superheating_lt': 0,
                  't_external_env': 308.15,
                  't_internal_env_ht': 278.15,
                  't_internal_env_lt': 258.15,
                  'work': 900},
 'exergy_efficiency': 0.5682108721688544,
 'm': 0.002521376583472338,
 'm_ht': 0.0002521376583472337,
 'm_lt': 0.002269238925125104,
 'point_1': {'H': 1586702.0214511054,
             'P': 190026.10012116132,
             'Q': -1.0,
             'S': 6400.779445580925,
             'T': 254.64370729430541,
             'refrigerant': 'NH3'},
 'point_2': {'H': 1943649.8910789562,
             

In [53]:
# Otimizacao do f

def determine_if_reached_threshold(value, lower, upper):
    reached_threshold = 0
    if value <= lower:
        reached_threshold = 1
        value = lower
    elif value >= upper:
        reached_threshold = 1
        value = upper  
    return value, reached_threshold

def calculate_next_x(input_values, x, current_cycle, delta, alpha, lower_threshold, upper_threshold):
    current_x = input_values[x]
    input_values_x = input_values
    input_values_x[x] += delta
    gradient_cycle_x = calculate_cycle(input_values_x)
    gradient_x = (gradient_cycle_x['cop'] - current_cycle['cop']) / delta
    next_x = current_x + alpha * gradient_x
    next_x, reached_threshold_x = determine_if_reached_threshold(next_x, lower_threshold, upper_threshold)
    return next_x, reached_threshold_x
 
# Calcula proximo k

def calculate_next_all(opt_input_values, delta, alpha):
      
    current_cycle = calculate_cycle(opt_input_values)

    # Next superheating_ht
    next_superheating_ht, reached_threshold_superheating_ht = calculate_next_x(opt_input_values, 
                                                                               'superheating_ht', 
                                                                               current_cycle, 
                                                                               delta, 
                                                                               alpha, 
                                                                               opt_input_values['lower_threshold'], 
                                                                               opt_input_values['upper_threshold'])

    # Next superheating_lt
    next_superheating_lt, reached_threshold_superheating_lt = calculate_next_x(opt_input_values, 
                                                                               'superheating_lt', 
                                                                               current_cycle, 
                                                                               delta, 
                                                                               alpha, 
                                                                               opt_input_values['lower_threshold'], 
                                                                               opt_input_values['upper_threshold'])

    # Next subcooling
    next_subcooling, reached_threshold_subcooling = calculate_next_x(opt_input_values, 
                                                                     'subcooling', 
                                                                     current_cycle, 
                                                                     delta, 
                                                                     alpha, 
                                                                     opt_input_values['lower_threshold'], 
                                                                     opt_input_values['upper_threshold'])
   
    # Next f
    next_f, reached_threshold_f = calculate_next_x(opt_input_values, 'f', current_cycle, delta, alpha, 0.1, 0.9)
        
    opt_input_values['superheating_ht'] = next_superheating_ht
    opt_input_values['superheating_lt'] = next_superheating_lt
    opt_input_values['subcooling'] = next_subcooling
    opt_input_values['f'] = next_f
    sum_of_threshold_reached =  reached_threshold_superheating_ht + reached_threshold_superheating_lt \
        + reached_threshold_subcooling + reached_threshold_f
    
    next_cycle = calculate_cycle(opt_input_values)
    
    return sum_of_threshold_reached, current_cycle, next_cycle

def optimize_with_cop():
    n = 0
    error = 1
    while n < 4 and error >= 10**(-7):
        n, current_cycle, next_cycle = calculate_next_all(opt_input_values, 10**(-4), 5)
        error = (next_cycle['cop'] - current_cycle['cop'])/((next_cycle['cop'] + current_cycle['cop'])/2)
    print_cycle = calculate_cycle(opt_input_values)
    print('#'*30)
    print('f: ', print_cycle['cycle_inputs']['f'])
    print('subcooling: ', print_cycle['cycle_inputs']['subcooling'])
    print('superheating_ht: ', print_cycle['cycle_inputs']['superheating_ht'])
    print('superheating_lt: ', print_cycle['cycle_inputs']['superheating_lt'])
    print('cop: ', print_cycle['cop'])
    print('exergy_efficiency: ', print_cycle['exergy_efficiency'])
    print('#'*30)

# Variaveis de entrada para otimizacao
opt_input_values = {
    't_external_env': 35 + 273.15,
    't_internal_env_ht': 5 + 273.15,
    't_internal_env_lt': -15 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'work': 900,
    'f': 0.5,
    'isentropic_efficiency': 0.7,
    'subcooling': 5,
    'superheating_ht': 5,
    'superheating_lt': 5,
    'upper_threshold': 10,
    'lower_threshold': 0,
    'refrigerant': 'NH3'
}
    
optimize_with_cop()

##############################
f:  0.9
subcooling:  10
superheating_ht:  0
superheating_lt:  0
cop:  2.398227631280447
exergy_efficiency:  0.4436211676485282
##############################
